In [32]:
import pandas as pd
import time
import websocket
import json, pprint, talib 
import csv
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from binance.client import Client
from binance.enums import *
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from pytrends.request import TrendReq #as UTrendReq
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

import os
import sqlite3 as sl
from sqlite3 import Error
import pymysql

cmc = CoinMarketCapAPI('a01ace6b-f43c-439e-9fca-dc7f73e2a648')
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)


In [33]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [34]:
sql_create_crypto_table = """CREATE TABLE IF NOT EXISTS crypto (
                                id int(7) NOT NULL auto_increment,
                                name text(30) NOT NULL,
                                name_searchable text(30),
                                ticker varchar(11) NOT NULL,
                                symbol varchar(6),
                                cmc_id int(7),
                                category text(255),
                                date_added text(30),
                                tags text(255),
                                twitter_username text(50),
                                self_reported_tags text(255),
                                subreddit text(50),
                                PRIMARY KEY(id)
                            );"""

sql_create_crypto_price_table = """CREATE TABLE IF NOT EXISTS crypto_price (
                    crypto_id int(7),
                    date datetime NOT NULL,
                    open decimal(30, 20) NOT NULL,
                    high decimal(30, 20) NOT NULL,
                    low decimal(30, 20) NOT NULL,
                    close decimal(30, 20) NOT NULL,
                    volume integer(20),
                    PRIMARY KEY(crypto_id, date),
                    FOREIGN KEY (crypto_id) REFERENCES crypto (id)
                );"""

# sql_create_crypto_trend_table = """CREATE TABLE IF NOT EXISTS crypto_trend (
#                     crypto_id int(7),
#                     date datetime NOT NULL,
#                     hype decimal(5,2) NOT NULL,
#                     PRIMARY KEY(crypto_id, date),
#                     FOREIGN KEY (crypto_id) REFERENCES crypto (id)
#                 );"""

sql_create_crypto_trend_table = """CREATE TABLE IF NOT EXISTS crypto_trend (
                    crypto_id int(7),
                    date_request datetime NOT NULL,
                    date datetime NOT NULL,
                    hype decimal(5,2) NOT NULL,
                    PRIMARY KEY(crypto_id, date),
                    FOREIGN KEY (crypto_id) REFERENCES crypto (id)
                );"""                

sql_create_crypto_trades_table = """CREATE TABLE IF NOT EXISTS crypto_trades (
                    crypto_id int(7),
                    date_buy datetime NOT NULL,
                    price_buy decimal(30, 20) NOT NULL,
                    date_sell datetime NOT NULL,
                    price_sell decimal(30, 20) NOT NULL,
                    trade_return decimal(4, 3) NOT NULL,
                    trade_time varchar(8) NOT NULL,
                    PRIMARY KEY(crypto_id, date_buy),
                    FOREIGN KEY (crypto_id) REFERENCES crypto (id)
                );"""                
 

#connect to aws rds
conn = None
try:
    conn = pymysql.connect(host='coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com', user='admin', password='0323A8E3DB', database='crypto_db')
    #CREATE DATABASE movies;
    curr = conn.cursor()
    curr.execute('SET FOREIGN_KEY_CHECKS = 1')
except Error as e:
    print(e)

# create tables
if conn is not None:
    # create crypto table
    create_table(conn, sql_create_crypto_table)
    # create artifact table
    create_table(conn, sql_create_crypto_price_table)
    # create artifact_price table
    create_table(conn, sql_create_crypto_trend_table)
else:
    print("Error! cannot create the database connection.")

In [35]:
#filter out leverage tokens, non usd tokens, 
all_pairs = pd.DataFrame(client.get_ticker())
relev = all_pairs[all_pairs.symbol.str.endswith('USDT')]
non_lev_list = relev[ ~( (relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('BEAR')) | (relev.symbol.str.contains('BULL')) ) ]
#filter out stable coins
non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
#remove BCHSV, PAX, STRAT, XZC
remove =  [
'BCHSVUSDT',
'PAXUSDT',
'STRATUSDT',
'XZCUSDT',
'AUDUSDT',
'EURUSDT',
'GBPUSDT',
'BCCUSDT',
'BCHABCUSDT',
'BUSDUSDT',
'BTGUSDT',
'ERDUSDT',
'LENDUSDT',
'MCOUSDT',
'GXSUSDT',
'IOTAUSDT',
'NPXSUSDT',
'PAXGUSDT',
'VENUSDT',
'TUSDUSDT',
'STORMUSDT',
'SUSDUSDT',
'BKRWUSDT',
'BTTUSDT',
'BZRXUSDT'
]

namechange = {
'Terra':'LUNA coin',
'Kyber Network Crystal v2': 'Kyber Network',
'Drep [new]':'Drep',
'PERL.eco':'Perl coin',
'Fetch.ai':'Fetch coin',
'Quant':'Quant Network',
'Polygon':'polygon matic',
'VITE':'vite coin',
'Bounce Finance Governance Token': 'Bounce finance',
'S.S. Lazio Fan Token': 'Lazio Fan Token',
'Anchor Protocool' : 'anc coin'
}


for ticker in remove:
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == ticker].index )
non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.askPrice == '0.00000000'].index )
token_tickers = non_stable_rm.sort_values(by='symbol', key=lambda x: x.str.split('USDT').str[0])
#remove different tickers between binance and cmc
symbols=list(token_tickers['symbol'].str.slice(0,-4))
symbols[symbols.index('BTTC')] = 'BTTOLD'
#take symbols into commas seperated list for cmc api
symbolslist = ",".join(symbols)
cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)
names_info = cmcinfo.data
#removing punctuation from names
names = []
for ind in names_info:
    names.append(names_info[ind]['name'].replace('[', '').replace(']', '').replace('(', '') .replace(')', ''))
#taking binance tickers inserting into cmc dataframe
names_info = pd.DataFrame(names_info).T
names_info.insert(1, "ticker", list(token_tickers['symbol']))
names_info = names_info.rename({'id': 'cmc_id'}, axis=1)
alltokens = names_info[['name','ticker','symbol','cmc_id','category','date_added','tags', 'twitter_username','self_reported_tags', 'subreddit']]
alltokens['tags'] = alltokens['tags'].str.join(', ')
alltokens['self_reported_tags'] = alltokens['self_reported_tags'].str.join(', ')
alltokens = alltokens.reset_index(drop=True)
# alltokens = alltokens.reset_index(drop=False)
# alltokens = alltokens.rename({'index': 'id'}, axis=1)
alltokens.insert(2, 'name_searchable', alltokens['name'])
#replace searchable names
alltokens['name_searchable'].replace(to_replace=namechange, inplace=True)
crypto_table = alltokens

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_19888/1967828573.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_19888/1967828573.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alltokens['tags'] = alltokens['tags'].str.join(', ')
C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_19888/1967828573.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [36]:
from sqlalchemy import create_engine
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

crypto_table.to_sql("crypto", engine, if_exists="append", index=False)